In [ ]:
import etherscan
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt # importing matplotlib
import numpy as np # importing numpy
import time

from dotenv import load_dotenv
from datetime import datetime
from web3 import Web3
from os import getenv
from tqdm import tqdm
from decimal import Decimal

In [ ]:
load_dotenv()


es = etherscan.Client(api_key=getenv('ETHERSCAN'))
w3 = Web3(Web3.HTTPProvider(getenv('INFURA')))

w3.isConnected()

In [ ]:
# address book
carlisle = '0xd791427e372186E90637caF1D6D6AFb054BeeC7A'
rarible = '0xcd4EC7b66fbc029C116BA9Ffb3e59351c20B5B06'
opensea = '0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b'
foundation = '0xcDA72070E455bb31C7690a170224Ce43623d0B6f'
superrare = '0x41A322b28D0fF354040e2CbC676F0320d8c8850d'
knownorigin = '0x848b0EA643e5a352D78e2c0c12A2dD8c96feC639'

In [ ]:
def get_balance(wallet):
    wei = w3.eth.get_balance(wallet)
    return w3.fromWei(wei, 'ether')

In [ ]:
get_balance(carlisle)

In [ ]:
def mp_transactions(address):
    columns = ['timestamp', 'block_number', 'from', 'to', 'input', 'hash', 'value', 'gas', 
                'gas_price', 'gas_used', 'nonce', 'confirmations', 'is_error', 'tx_receipt_status', 
                'transaction_index', 'cumulative_gas_used', 'block_hash']

    transactions = []

    latest_block = w3.eth.get_block('latest')
    end_block = latest_block['number']
    start_block = end_block - (6500 * 5)

    for l in tqdm(es.get_transactions_by_address(address, start_block=start_block, end_block=end_block, limit=9999)):
        time.sleep(.2)
        transactions.append(l)

    df = pd.DataFrame(transactions, columns=columns)
    df['timestamp'] = df['timestamp'].apply(datetime.fromtimestamp)
    df = df[df['value'] != 0]
    df = df.reset_index(drop=True)
    df['value'] = [w3.fromWei(x, 'ether') for x in df['value']]
    return df

In [ ]:
rarible_df = mp_transactions(rarible)

In [ ]:
rarible_df['value'].mean()

In [ ]:
rarible_df

In [ ]:
fig = go.Figure()

value = rarible_df['value']
timestamp = rarible_df['timestamp']
a_hash = rarible_df['hash']
mean = rarible_df['value'].mean()

fig.add_trace(go.Scatter(y=value, x=timestamp, name=str(a_hash), mode='markers', text=rarible_df.index,
                marker=dict(size=value.astype(float), color=value, colorscale='Cividis', showscale=True, line=dict(width=.5,
                                        color='grey'))))

fig.add_shape(type='line', x0=min(timestamp), y0=30, x1=max(timestamp), y1=30,
                line=dict(color='Red') ,xref='x', yref='y')

fig.update_layout(yaxis_title="", xaxis_title="<i>contract: 0xcd4EC7b66fbc029C116BA9Ffb3e59351c20B5B06</i>", width=600, height=900, yaxis=dict(zeroline=False),
                    template='seaborn', legend_title="Comparison", paper_bgcolor='rgba(0,0,0,0)', 
                    font=dict(family="Tahoma", size=10), plot_bgcolor='rgba(0,0,0,0)',
                    title=("Rarible<br>" +
                    "<i>Value of transactions including bids and sales</i>"))

# Add images
fig.add_layout_image(dict(source="gradient.png", xref="paper", yref="paper",
                        x=-1.5, y=1.2, sizex=6, sizey=6, layer="below"))

fig.add_annotation(x=rarible_df['timestamp'].iloc[2170], y=70.5,
            text="Beeple", showarrow=False, arrowhead=1)

fig.update_shapes(dict(xref='x', yref='y'))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_traces(marker_showscale=False)
fig.show()

In [ ]:
def hist_for_count(data):
    plt.style.use("fast")
    plt.figure(figsize=(21,14), facecolor=('#fff9f4')) # Make it 14x7 inch

    x = data # normal distribution
    n, bins, patches = plt.hist(x, bins=80, facecolor='#2ab0ff', edgecolor='#e0e0e0', linewidth=0.5, alpha=0.7)
    n = n.astype('int') # it MUST be integer

    # Good old loop. Choose colormap of your taste
    for i in range(len(patches)):
        patches[i].set_facecolor(plt.cm.viridis(n[i]/max(n)))

    # Add title and labels with custom font sizes
    plt.title('', fontsize=12)
    plt.xlabel('', fontsize=8)
    plt.ylabel('', fontsize=8)
    plt.grid(b=None)
    plt.savefig('foo.png')
    plt.show();
    return

In [ ]:
hist_for_count(foundation_df['timestamp'])